# Initialisation

## Importation

In [7]:
import cv2
import math
import sklearn
import random
import progressbar
import numpy as np
# import moviepy.editor as mpe
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from time import time
from scipy import misc
from scipy import ndimage
from IPython import display
from PIL import Image, ImageOps
from IPython.display import YouTubeVideo
from scipy.ndimage.filters import convolve

from sklearn.cluster import KMeans
import torch
from torch import nn

## Database

In [13]:
with ZipFile('../HandGestureDataset_SHREC2017_temp.zip', 'r') as zipObj:
  zipObj.extractall()
  zipObj.close()

root_datase = './HandGestureDataset_SHREC2017_temp'

In [14]:
def open_data(idx_gesture,
              idx_subject,
              idx_finger,
              idx_essai):

  # Path of the gesture
  path_gesture = '{0}/gesture_{1}/finger_{2}/subject_{3}/essai_{4}/'.format(root_datase, idx_gesture+1, idx_finger+1, idx_subject+1, idx_essai+1)

  if os.path.isdir(path_gesture):
    return True, np.loadtxt(path_gesture + '/skeletons_image.txt')
  else:
    return False, None

Ground_truth = []
X = []
for idx_gesture in range(14):
  for idx_subject in range(28):
    for idx_finger in range(2):
      for idx_essai in range(10):
        Exist, x = open_data(idx_gesture, idx_subject, idx_finger, idx_essai)
        if not(Exist) :
          break
        X.append(x)
        Ground_truth.append(idx_gesture)

# Réseaux de neurones récurrents

## Data formating

In [15]:
max = 0
for x in X:
  if max < len(x):
    max = len(x)

Data = np.zeros((len(X), max, X[0].shape[1]))
for i in range(len(X)):
  Data[i, :X[i].shape[0], :] = np.reshape(X[i], (X[i].shape[0], X[i].shape[1]), order = 'F')

Target = np.zeros((len(Ground_truth), 14))
for i in range(len(Ground_truth)):
  Target[i, Ground_truth[i]] = 1

dataSize, seqSize, inputSize = Data.shape
outputSize = Target.shape[1]
trainSize = int(dataSize * 0.9)

ind_train = np.random.choice(dataSize, trainSize, replace = False)
train_data = torch.from_numpy(Data[ind_train])
train_target = torch.from_numpy(Target[ind_train])

ind_test = np.delete(np.arange(dataSize), ind_train)
test_data = torch.from_numpy(np.array(Data[ind_test]))
test_target = Target[ind_test]

## Model definition

In [38]:
class RNN_classifier(nn.Module):
  def __init__(self, inputSize, seqSize, outputSize, device="cpu"):
    super().__init__()
    
    self.f = nn.Tanh().to(device) # Hyperparameter 
    self.f_out = nn.Softmax(dim = 1).to(device) # nn.Sigmoid() # Hyperparameter 

    self.inputSize = inputSize
    self.R_Size = 100 # Hyperparameter
    self.Q_Size = 100 # Hyperparameter
    self.A = [0, 0.25, 0.5, 0.95] # Hyperparameter
    self.H_Size = len(self.A) * self.Q_Size
    self.O_Size = 100 # Hyperparameter
    self.seqSize = seqSize
    self.outputSize = outputSize
    self.loss = nn.BCELoss()


    self.R = nn.Linear(self.H_Size, self.R_Size)
    self.Q = nn.Linear(self.inputSize + self.R_Size, self.Q_Size)
    self.O = nn.Linear(self.H_Size, self.O_Size)
    self.flat = nn.Flatten()
    self.Output = nn.Linear(self.O_Size * self.seqSize, self.outputSize)

    self.to(device)

  def forward(self, In):
    batchSize = In.shape[0]
    Out = torch.zeros((batchSize, self.seqSize, self.O_Size))
    Ha = torch.zeros((batchSize, self.seqSize, len(self.A), self.Q_Size))

    for t in range(self.seqSize):
      H = torch.reshape(Ha[:, t - 1].clone(), (batchSize, self.H_Size))
      Rt = self.f(self.R(H))
      print(In[:, t].device, Rt.device)
      Qt = self.f(self.Q(torch.cat((In[:, t], Rt), 1)))
      for a, Alpha in enumerate(self.A) :
        Ha[:, t, a] = Alpha * Ha[:, t - 1, a].clone() + (1 - Alpha) * Qt
      H = torch.reshape(Ha[:, t].clone(), (batchSize, self.H_Size))
      Out[:, t] = self.f(self.O(H))
    
    return self.f_out(self.Output(self.flat(Out)))



## Training

In [36]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
    print("Using cuda")
else :
    print("Using cpu")

model = RNN_classifier(inputSize, seqSize, outputSize, device=device)
model.train()

lossHistory = []
outputs = []
target = []
test_Score = []

adresse = './checkpoints'

Using cuda


In [39]:
batchSize = 100
learningRate = 0.0001
epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

affichage = 5
moyennage = 10
saving = 10
bar = progressbar.ProgressBar(maxval=epochs)
bar.start()
bar.update(0)

for epoch in range(epochs):

  batch = np.random.choice(trainSize, batchSize)
  output = model.forward(train_data[batch].float().to(device))
  loss = model.loss(output, train_target[batch].float())
  model.zero_grad()
  loss.backward()
  optimizer.step()

  outputs.append(output.to("cpu"))
  target.append(train_target[batch].to("cpu"))
  lossHistory.append(loss.to("cpu"))

  test_output = model.forward(test_data.float().to(device)).detach().numpy()
  test_Score.append(np.mean(np.argmax(test_output.to("cpu"), axis=1) == np.argmax(test_target, axis=1))*100)

  if (len(test_Score) - 1) % saving == 0 :
    path = adresse + '/{}.pt'.format(len(test_Score) - 1)
    torch.save({'epoch': len(test_Score) - 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'test_Score': test_Score[-1]}, path)
    files.download(path)
  if (epoch + 1) % affichage == 0 :
    display.clear_output(wait=True)
    plt.clf()

    fig, axs = plt.subplots(2, 1, figsize=(16, 18))
    axs[0].plot(lossHistory)
    # axs[0].plot(np.convolve(lossHistory, np.ones(moyennage)/moyennage)[moyennage - 1 : - moyennage + 1])
    # axs[0].legend(['loss', 'loss moyen'])
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')

    axs[1].plot(test_Score)
    axs[1].set_title('Réussite du set de test')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Score  (%)')
    plt.grid(True)
    
    display.display(plt.gcf())

  torch.cuda.empty_cache()
  bar.update(epoch + 1)

display.clear_output(wait=True)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'mat1' in call to _th_addmm